In [ ]:
import logging
# 配置日志
logging.basicConfig(level=logging.INFO)
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Replace 'your_model_file' with the path to the model file you downloaded.
model_id = "/Users/jet.xu/Mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id, device_map="mps", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="mps", trust_remote_code=True, torch_dtype=torch.float16)

In [ ]:
# Composing the instruction with the necessary template
instruction = "<s> [INST] {} [/INST]".format("请详细解释一下什么是MOE混合专家大模型的架构和原理，请使用中文回答。")
inputs = tokenizer(instruction, return_tensors="pt")
inputs.to("mps")
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, temperature=0.9, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))